In [ ]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html;
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html;
!pip install -q torch-geometric;

     |████████████████████████████████| 8.0 MB 5.3 MB/s 
     |████████████████████████████████| 2.9 MB 5.0 MB/s 
     |████████████████████████████████| 325 kB 5.4 MB/s 
     |████████████████████████████████| 407 kB 35.6 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 


In [ ]:
import torch
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch.nn as nn
from torch.nn.modules.module import Module

import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR,StepLR

import numpy as np
import pandas as pd
import pickle as pkl
import sys
import networkx as nx
import scipy.sparse as sp
import math
import matplotlib.pyplot as plt
import time
from time import perf_counter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Utils functions: visualization


In [ ]:
def visualize(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])

    if torch.is_tensor(h):
        h = h.detach().cpu().numpy()
        plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
        if epoch is not None and loss is not None:
            plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    else:
        nx.draw_networkx(h, pos=nx.spring_layout(h, seed=42), with_labels=False,
                         node_color=color, cmap="Set2")
    plt.show()

def normalize_adjacency_matrix(A, I):
  """
  Creating a normalized adjacency matrix with self loops.
  :param A: Sparse adjacency matrix.
  :param I: Identity matrix.
  :return A_tile_hat: Normalized adjacency matrix."""
  
  A_tilde = I
  degrees = A_tilde.sum(axis=0)[0].tolist()
  D = sp.diags(degrees, [0])
  D = D.power(-0.5)
  A_tilde_hat = D.dot(A_tilde).dot(D)
  return A_tilde_hat

def normalize(mx):
  """Row-normalize sparse matrix ---> Node features"""
  rowsum = np.array(mx.sum(1))
  r_inv = np.power(rowsum, -1).flatten()
  r_inv[np.isinf(r_inv)] = 0.
  r_mat_inv = sp.diags(r_inv)
  mx = r_mat_inv.dot(mx)
  return mx

def normalizemx(mx):
  """Normalization for Scattering GCN"""
  degrees = mx.sum(axis=0)[0].tolist()
  #    print(degrees)
  D = sp.diags(degrees, [0])
  D = D.power(-1)
  mx = mx.dot(D)
  return mx


def scattering1st(spmx,order):

  I_n = sp.eye(spmx.shape[0])
  adj_sct = 0.5*(spmx+I_n) # P = 1/2 * (I + WD^-1)
  adj_power = adj_sct
  adj_power = sparse_mx_to_torch_sparse_tensor(adj_power).cuda()
  adj_sct = sparse_mx_to_torch_sparse_tensor(adj_sct).cuda()
  I_n = sparse_mx_to_torch_sparse_tensor(I_n)
  if order>1:
    for i in range(order-1):
      # Generating P^(2^(k-1))
      adj_power = torch.spmm(adj_power,adj_sct.to_dense())
      print('Generating SCT')
    # Generating. final scattering of order K -> (I - P^(2^(k-1))) * P^(2^(k-1))
    adj_int = torch.spmm((adj_power-I_n.cuda()),adj_power)
  else:
    # Generating. final scattering of order K -> (I - P^(2^(k-1))) * P^(2^(k-1))
    adj_int = torch.spmm((adj_power-I_n.cuda()),adj_power.to_dense())
  return adj_int


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
  """Convert a scipy sparse matrix to a torch sparse tensor."""
  sparse_mx = sparse_mx.tocoo().astype(np.float32)
  indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
  values = torch.from_numpy(sparse_mx.data)
  shape = torch.Size(sparse_mx.shape)
  return torch.sparse.FloatTensor(indices, values, shape)


def parse_index_file(filename):
  #Parse index file.
  index = []
  for line in open(filename):
      index.append(int(line.strip()))
  return index

def accuracy(output, labels):
  preds = output.max(1)[1].type_as(labels)
  correct = preds.eq(labels).double()
  correct = correct.sum()
  return correct / len(labels)

# Preprocessing: Importing datasets

Importing the datasets, split into training, validation and testing, normalizing it, getting the adjacency matrix, the scattering matrices, features matrix, index of nodes.

In [ ]:
def load_citation(dataset_str="citeseer", normalization="AugNormAdj", cuda=True):
  """  
  Load Citation Networks Datasets.
  """
  names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
  objects = []
  for i in range(len(names)):
    with open("/content/drive/MyDrive/THESIS/Databases/data/ind.{}.{}".format(dataset_str.lower(), names[i]), 'rb') as f:
      if sys.version_info > (3, 0):
          objects.append(pkl.load(f, encoding='latin1'))
      else:
          objects.append(pkl.load(f))

  x, y, tx, ty, allx, ally, graph = tuple(objects)
  test_idx_reorder = parse_index_file("/content/drive/MyDrive/THESIS/Databases/data/ind.{}.test.index".format(dataset_str))
  test_idx_range = np.sort(test_idx_reorder)

  if dataset_str == 'citeseer':
    # Fix citeseer dataset (there are some isolated nodes in the graph)
    # Find isolated nodes, add them as zero-vecs into the right position
    test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
    tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
    tx_extended[test_idx_range-min(test_idx_range), :] = tx
    tx = tx_extended
    ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
    ty_extended[test_idx_range-min(test_idx_range), :] = ty
    ty = ty_extended

  features = sp.vstack((allx, tx)).tolil()
  features[test_idx_reorder, :] = features[test_idx_range, :]
  adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))
  adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
  labels = np.vstack((ally, ty))
  labels[test_idx_reorder, :] = labels[test_idx_range, :]


  idx_test = test_idx_range.tolist()
  idx_train = range(len(y))
  idx_val = range(len(y), len(y)+500)

  #   take from https://github.com/tkipf/pygcn/blob/master/pygcn/utils.py
  #    idx_train = range(140)
  #    idx_val = range(200, 500)
  #    idx_test = range(500, 1500)


  labels = torch.LongTensor(labels)
  labels = torch.max(labels, dim=1)[1]
  idx_train = torch.LongTensor(idx_train)
  idx_val = torch.LongTensor(idx_val)
  idx_test = torch.LongTensor(idx_test)

  features = normalize(features)
  A_tilde = normalize_adjacency_matrix(adj,sp.eye(adj.shape[0]))
  adj = normalizemx(adj)
  features = torch.FloatTensor(np.array(features.todense()))
  print('Loading')
  #adj_sct1 = scattering1st(adj,1) ## psi_1 = P(I-P)
  #adj_sct2 = scattering1st(adj,2) # psi_2 = P^2(I-P^2)
  #adj_sct4 = scattering1st(adj,4) # psi_3 = P^4(I-P^4)
  adj = sparse_mx_to_torch_sparse_tensor(adj)
  A_tilde = sparse_mx_to_torch_sparse_tensor(A_tilde)
  return adj,A_tilde,features, labels, idx_train, idx_val, idx_test


In [ ]:
adj,A_tilde,features, labels, idx_train, idx_val, idx_test = load_citation()

Loading


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power


# MODELS

First the convolutional structure is defined to finally being called in a nn Module. 

In [ ]:
class GraphConvolution(Module):
    """
    Simple GCN layer
    """
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
         
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self,nfeat,n_class, hidden_channels, degree):
        super(GCN, self).__init__()
        torch.manual_seed(1234567)
        self.conv1 = GraphConvolution(nfeat, hidden_channels)
        self.lin1 = nn.Linear(hidden_channels,n_class)
        self.d = degree

    def forward(self, x, adj):
        x = self.conv1(x, adj)
        x = x.relu()
        for i in range(self.d):
          x = torch.spmm(adj, x)
        return self.lin1(x)

model = GCN(features.shape[1],labels.max().item() + 1, hidden_channels=16, degree=2)
print(model)

GCN(
  (conv1): GraphConvolution (3703 -> 16)
  (lin1): Linear(in_features=16, out_features=6, bias=True)
)


# Execution of the overall model

Hyperparameter definition, model instatiated, and training and testing

In [ ]:
torch.manual_seed(42)

epochs = 200
lr = 0.01
cuda = torch.cuda.is_available()

if cuda:
    model = model.cuda()
    features = features.cuda()
    A_tilde = A_tilde.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()


optimizer = optim.Adam(model.parameters(),lr=lr)
criterion = torch.nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=50, gamma=0.9)

In [ ]:
import time
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 430})'''))

def acc1(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def train(x, adj, labels, idx_train):
  t = perf_counter()
  optimizer.zero_grad()
  out = model(x, adj)
  loss = criterion(out[idx_train], labels[idx_train])  # Compute the loss solely based on the training nodes.
  train_acc = acc1(out[idx_train], labels[idx_train])
  loss.backward()  # Derive gradients.
  optimizer.step()
  train_time = perf_counter()-t

  with torch.no_grad():
    model.eval()
    corrects = 0
    output = model(x, adj)
    val_acc = acc1(out[idx_val], labels[idx_val])
  return val_acc, train_time

def test(nfeat, adj, labels, idx_test):
  model.eval()
  out = F.softmax(model(nfeat, adj), dim=1)
  test_acc =  acc1(out[idx_test], labels[idx_test])
  return test_acc

global_acc = []
for i in range(5):
  acc_list = []
  loss_list = []
  for epoch in range(200):
    loss, h= train(features, A_tilde, labels, idx_train)
    accuracy = test(features, A_tilde, labels, idx_test)
    acc_list.append(accuracy)
    loss_list.append(loss)
  total_acc = accuracy
  global_acc.append(total_acc.item())

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
print(global_acc)
accuracy = np.mean(global_acc)
print('Acc after 5 runs:', accuracy)

[0.5, 0.503, 0.493, 0.492, 0.492]
Acc after 5 runs: 0.496
